In [1]:
import os
import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten, Dropout
from keras.layers.core import Dense
from keras.datasets import cifar10
from keras.optimizers import RMSprop
from keras.callbacks import TensorBoard, ModelCheckpoint

Using TensorFlow backend.


In [2]:
# data prepararion
class CIFAR10Dataset():
    def __init__(self):
        
        # image shape
        self.image_shape = (32,32,3)
        self.num_classes = 10
        
    def get_batch(self):
        (X_train, y_train), (X_test, y_test) = cifar10.load_data()
        X_train, X_test = [self.preprocess(d) for d in [X_train, X_test]]
        y_train, y_test = \
        [self.preprocess(d, label_data=True) for d in [y_train, y_test]]
        return X_train, y_train, X_test, y_test
      
        
    def preprocess(self, data, label_data=False):
        # for labels
        if label_data:
            # convert class vectors to binary class matrices
            data = keras.utils.to_categorical(data, self.num_classes)
            
        # for pixels data
        else:
            data = data.astype('float32')
            # convert the value to 0~1 scale (pixel's max value = 255)
            data = data / 255
            # add data set length to top
            shape = (data.shape[0], ) + self.image_shape
            data = data.reshape(shape)
            
        return data

In [3]:
# LeNet network
def network(input_shape, num_classes):
    model = Sequential()
    
    # extract image features by convolution and max pooling layers
    model.add(Conv2D(32, kernel_size=3, padding='same',
                    input_shape=input_shape, activation='relu'))
    
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    # classifiy the class by fully-connected layers
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    return model

In [4]:
class Trainer():
    def __init__(self, model, loss, optimizer):
        self._target = model
        self._target.compile(loss=loss, optimizer=optimizer,metrics=['accuracy'])
        self.verbose = 1
        self.log_dir = os.path.join(os.path.dirname(os.path.abspath('__file__')),'logdir')
        self.model_file_name = 'model_file.hdf5'
        
    def train(self, X_train, y_train, batch_size, epochs, validation_split):
        if os.path.exists(self.log_dir):
            import shutil
            # remove previous execution
            shutil.rmtree(self.log_dir)
        else:
            pass
        os.mkdir(self.log_dir)
        
        self._target.fit(
            X_train, y_train,
            batch_size=batch_size,
            epochs=epochs, 
            validation_split=validation_split,
            callbacks=[TensorBoard(log_dir=self.log_dir),
                      ModelCheckpoint(os.path.join(self.log_dir,
                                    self.model_file_name),
                                      save_best_only=True)],
             verbose=self.verbose
        )

In [5]:
dataset = CIFAR10Dataset()

In [6]:
# make model
model = network(dataset.image_shape, dataset.num_classes)

W0829 20:14:32.276090 23332 deprecation_wrapper.py:119] From C:\Users\gudeg\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0829 20:14:32.304141 23332 deprecation_wrapper.py:119] From C:\Users\gudeg\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0829 20:14:32.308092 23332 deprecation_wrapper.py:119] From C:\Users\gudeg\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0829 20:14:32.348097 23332 deprecation_wrapper.py:119] From C:\Users\gudeg\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0829 20:14:32.356128 23332 deprecation_wrapper.py:119] From C:\Users\gudeg\Ana

In [10]:
# prepare data
X_train, y_train, X_test, y_test = dataset.get_batch()

# train the model
trainer = Trainer(model, loss='categorical_crossentropy',
                 optimizer=RMSprop())

trainer.train(X_train, y_train, batch_size=128, epochs=13, validation_split=0.2)

Train on 40000 samples, validate on 10000 samples
Epoch 1/13
40000/40000 [==============================] - 85s 2ms/step - loss: 1.0370 - acc: 0.6368 - val_loss: 0.9211 - val_acc: 0.6751
Epoch 2/13
40000/40000 [==============================] - 79s 2ms/step - loss: 0.9843 - acc: 0.6542 - val_loss: 0.8869 - val_acc: 0.6893
Epoch 3/13
40000/40000 [==============================] - 85s 2ms/step - loss: 0.9513 - acc: 0.6666 - val_loss: 0.9651 - val_acc: 0.6659
Epoch 4/13
40000/40000 [==============================] - 78s 2ms/step - loss: 0.9194 - acc: 0.6755 - val_loss: 0.8588 - val_acc: 0.6989
Epoch 5/13
40000/40000 [==============================] - 88s 2ms/step - loss: 0.8916 - acc: 0.6904 - val_loss: 0.8382 - val_acc: 0.7092
Epoch 6/13
40000/40000 [==============================] - 83s 2ms/step - loss: 0.8690 - acc: 0.6949 - val_loss: 0.8180 - val_acc: 0.7156
Epoch 7/13
40000/40000 [==============================] - 84s 2ms/step - loss: 0.8533 - acc: 0.7028 - val_loss: 0.7839 - val_acc

In [11]:
# show result
score = model.evaluate(X_test, y_test, verbose=0)

In [12]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.7907178189277649
Test accuracy: 0.7308
